In [3]:
import os
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pvlib
from pvlib import location

# ---------- PHẦN 1: TÍNH TOÁN DỮ LIỆU CLEAR SKY ----------
# Define date range for GMT+7 (Asia/Saigon)
start_date = pd.Timestamp('2025-01-01 00:00:00', tz='Asia/Saigon')
end_date = pd.Timestamp('2025-03-14 23:59:59', tz='Asia/Saigon')

lat, lon = 10.76477848, 106.3148294
site = location.Location(lat, lon, tz='Asia/Saigon')

times = pd.date_range(start=start_date, end=end_date, freq='10min', tz='Asia/Saigon')
clearsky_data = site.get_clearsky(times).reset_index().rename(columns={'index': 'Time'})
clearsky_data.Time = pd.to_datetime(clearsky_data.Time).dt.tz_localize(None)
clearsky_data.set_index('Time', inplace=True)
clearsky_data = clearsky_data[['ghi']].rename(columns={'ghi': 'GHI_CS'})

# ---------- PHẦN 2: XỬ LÝ DỮ LIỆU GHI_CS ----------
power_df = clearsky_data.copy().reset_index()
power_df['Time'] = pd.to_datetime(power_df['Time'])
scaler = MinMaxScaler()
power_df['GHI_CS_normalized'] = scaler.fit_transform(power_df[['GHI_CS']])
power_lookup = power_df.set_index('Time')['GHI_CS_normalized'].to_dict()

# ---------- PHẦN 3: XỬ LÝ ẢNH VÀ TẠO PIPELINE ----------
SEQ_LENGTH = 24
PRED_LENGTH = 24
IMG_SIZE = (32, 32)
NUM_CHANNELS = 4
BATCH_SIZE = 128
EPOCHS = 30
START_DATE = datetime.datetime(2025, 1, 1)
END_DATE = datetime.datetime(2025, 1, 14)
PATH = r"Z:\Sky-image\namnvn\Data_process\MT Solarpark 1"

def load_and_process_image(path):
    """Load image from disk, resize và normalize về khoảng [-1,1]."""
    try:
        img = tf.io.read_file(path)
        # Sử dụng tf.image.decode_image có thể tự động xử lý định dạng
        img = tf.image.decode_image(img, channels=1, expand_animations=False)
        img = tf.image.resize(img, IMG_SIZE)
        img = (img / 127.5) - 1.0
        return img.numpy()
    except Exception as e:
        print(f"Lỗi load ảnh {path}: {e}")
        return np.zeros(IMG_SIZE + (1,), dtype=np.float32)

def preprocess_images(image_records_dict):
    """Ghép các kênh ảnh thành một chuỗi dữ liệu multi-channel."""
    print("\n🔄 Đang xử lý ảnh thành chuỗi dữ liệu multi-channel...")
    prefixes = ['b03_', 'b07_', 'b08_', 'b13_']
    sequences_images = []
    sequences_times = []
    
    # Tìm thời gian chung cho tất cả các kênh
    times = set.intersection(*(set(dt for _, dt in image_records_dict[prefix]) for prefix in prefixes))
    times = sorted(times)
    min_length = len(times) - SEQ_LENGTH - PRED_LENGTH + 1
    
    for i in range(max(0, min_length)):
        time_window = times[i:i+SEQ_LENGTH]
        if len(time_window) != SEQ_LENGTH:
            continue
        multi_channel_seq = []
        for prefix in prefixes:
            channel_seq = []
            for t in time_window:
                path = next((p for p, dt in image_records_dict[prefix] if dt == t), None)
                img = load_and_process_image(path) if path else np.zeros(IMG_SIZE + (1,), dtype=np.float32)
                channel_seq.append(img.squeeze())
            multi_channel_seq.append(channel_seq)
        # Stack thành (SEQ_LENGTH, IMG_SIZE[0], IMG_SIZE[1], NUM_CHANNELS)
        stacked_seq = np.stack(multi_channel_seq, axis=-1)
        sequences_images.append(stacked_seq)
        sequences_times.append(time_window[-1])
    
    sequences_images = np.array(sequences_images)
    print(f"✅ Đã xử lý {len(sequences_images)} chuỗi ảnh, shape: {sequences_images.shape}")
    return sequences_images, sequences_times

# Tạo image_records_dict sử dụng cấu trúc thư mục và tên file
date_dirs = [d.strftime("%Y%m%d") for d in pd.date_range(START_DATE, END_DATE)]
image_records_dict = {prefix: [] for prefix in ['b03_', 'b07_', 'b08_', 'b13_']}
for date_dir in date_dirs:
    date_path = os.path.join(PATH, date_dir)
    if os.path.exists(date_path):
        for time_dir in os.listdir(date_path):
            time_path = os.path.join(date_path, time_dir)
            if os.path.isdir(time_path):
                try:
                    dt = datetime.datetime.strptime(f"{date_dir}T{time_dir}", "%Y%m%dT%H%M")
                    for file in os.listdir(time_path):
                        for prefix in image_records_dict.keys():
                            if file.lower().startswith(prefix) and file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                                img_path = os.path.join(time_path, file)
                                image_records_dict[prefix].append((img_path, dt))
                except Exception as e:
                    print(f"Lỗi xử lý thư mục {time_path}: {e}")
                    continue
for prefix in image_records_dict:
    image_records_dict[prefix].sort(key=lambda x: x[1])
    
sequences_images_processed, sequences_times = preprocess_images(image_records_dict)

def prepare_dataset_with_processed_images(sequences_images_processed, sequences_times):
    """Ghép dữ liệu ảnh với chuỗi GHI_CS đã chuẩn hóa và tạo tf.data.Dataset."""
    print("\n🔄 Đang ghép dữ liệu ảnh với GHI_CS...")
    sequences_p_uoc = []
    targets = []
    valid_sequences_images = []
    valid_sequences_times = []
    for i, (img_seq, seq_time) in enumerate(zip(sequences_images_processed, sequences_times)):
        start_idx = i
        p_uoc_seq = [power_lookup.get(image_records_dict['b03_'][start_idx + j][1], 0.0) for j in range(SEQ_LENGTH)]
        target_dts = [image_records_dict['b03_'][start_idx + SEQ_LENGTH + j][1] for j in range(PRED_LENGTH)]
        target_powers = [power_lookup.get(dt, -1.0) for dt in target_dts]
        if all(power != -1.0 for power in target_powers):
            valid_sequences_images.append(img_seq)
            sequences_p_uoc.append(p_uoc_seq)
            targets.append(target_powers)
            valid_sequences_times.append(seq_time)
    
    ds = tf.data.Dataset.from_tensor_slices((
        (np.array(valid_sequences_images), np.array(sequences_p_uoc)),
        np.array(targets)
    ))
    # Có thể thêm shuffle nếu dataset lớn
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
    print(f"✅ Dataset đã sẵn sàng với {len(valid_sequences_images)} mẫu")
    return ds, valid_sequences_times

dataset, sequences_times_train = prepare_dataset_with_processed_images(sequences_images_processed, sequences_times)

# ---------- PHẦN 4: XÂY DỰNG MÔ HÌNH CẢI TIẾN ----------
def build_model():
    # Nhánh xử lý ảnh
    input_images = layers.Input(shape=(SEQ_LENGTH, IMG_SIZE[0], IMG_SIZE[1], NUM_CHANNELS))
    x = layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu'))(input_images)
    x = layers.TimeDistributed(layers.BatchNormalization())(x)
    x = layers.TimeDistributed(layers.MaxPooling2D(2, 2))(x)
    x = layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization())(x)
    x = layers.TimeDistributed(layers.MaxPooling2D(2, 2))(x)
    x = layers.TimeDistributed(layers.GlobalAveragePooling2D())(x)
    
    # Nhánh dữ liệu GHI_CS (chuỗi đã chuẩn hóa)
    input_p_uoc = layers.Input(shape=(SEQ_LENGTH,))
    p_uoc_features = layers.Reshape((SEQ_LENGTH, 1))(input_p_uoc)
    
    # Ghép các đặc trưng
    combined = layers.Concatenate(axis=-1)([x, p_uoc_features])
    # Sử dụng LSTM cải tiến (có thể thử dùng Bidirectional)
    lstm_out = layers.Bidirectional(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))(combined)
    # Có thể thêm một vài lớp Dense để cải thiện khả năng học
    dense_out = layers.Dense(64, activation='relu')(lstm_out)
    dense_out = layers.Dropout(0.3)(dense_out)
    output = layers.Dense(PRED_LENGTH)(dense_out)
    
    model = models.Model(inputs=[input_images, input_p_uoc], outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='mae', metrics=['mae'])
    model.summary()
    return model

model = build_model()

# ---------- PHẦN 5: HUẤN LUYỆN VỚI CALLBACKS ----------
def train_model(dataset):
    model = build_model()
    
    # Định nghĩa các callbacks
    early_stop = callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=1)
    checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor='loss', save_best_only=True, verbose=1)
    
    history = model.fit(dataset, epochs=EPOCHS, callbacks=[early_stop, reduce_lr, checkpoint], verbose=1)
    
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['mae'], label='MAE')
    plt.xlabel('Epoch')
    plt.ylabel('Giá trị')
    plt.title('Training Loss và MAE')
    plt.legend()
    plt.show()
    
    return model, history

model, history = train_model(dataset)

# ---------- PHẦN 6: KIỂM THỬ MÔ HÌNH ----------
def test_model(model, scaler, test_start_date, test_end_date):
    """Test the model with the given date range."""
    print(f"\n🔄 Testing model for period: {test_start_date} to {test_end_date}")
    global START_DATE, END_DATE
    START_DATE, END_DATE = test_start_date, test_end_date
    
    date_dirs = [d.strftime("%Y%m%d") for d in pd.date_range(START_DATE, END_DATE)]
    if not date_dirs:
        print("⚠️ No dates in the specified range!")
        return None, None
        
    test_image_records_dict = {prefix: [] for prefix in ['b03_', 'b07_', 'b08_', 'b13_']}
    for date_dir in date_dirs:
        date_path = os.path.join(PATH, date_dir)
        if os.path.exists(date_path):
            for time_dir in os.listdir(date_path):
                time_path = os.path.join(date_path, time_dir)
                if os.path.isdir(time_path):
                    try:
                        dt = datetime.datetime.strptime(f"{date_dir}T{time_dir}", "%Y%m%dT%H%M")
                        for file in os.listdir(time_path):
                            for prefix in test_image_records_dict.keys():
                                if file.lower().startswith(prefix) and file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                                    img_path = os.path.join(time_path, file)
                                    test_image_records_dict[prefix].append((img_path, dt))
                    except Exception as e:
                        print(f"Lỗi xử lý {time_path}: {e}")
                        continue
    for prefix in test_image_records_dict:
        test_image_records_dict[prefix].sort(key=lambda x: x[1])
    
    sequences_images_test, sequences_times_test = preprocess_images(test_image_records_dict)
    test_dataset, sequences_times = prepare_dataset_with_processed_images(sequences_images_test, sequences_times_test)
    
    # Lọc dữ liệu trong khoảng giờ 5h-19h
    filtered_indices = [i for i, t in enumerate(sequences_times) if 5 <= t.hour <= 19]
    if not filtered_indices:
        print("⚠️ Không có dữ liệu trong khoảng 5h-19h!")
        return None, None
    
    test_loss, test_mae = model.evaluate(test_dataset, verbose=1)
    print(f"✅ Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")
    
    predictions = model.predict(test_dataset)
    true_values = np.concatenate([y.numpy() for _, y in test_dataset], axis=0)
    true_values = scaler.inverse_transform(true_values)
    predictions = scaler.inverse_transform(predictions)
    
    filtered_true_values = true_values[filtered_indices]
    filtered_predictions = predictions[filtered_indices]
    filtered_times = [sequences_times[i] for i in filtered_indices]
    
    plt.figure(figsize=(15, 6))
    plt.plot(filtered_times, filtered_true_values[:, 0], label='Thực tế bước 1', color='blue')
    plt.plot(filtered_times, filtered_predictions[:, 0], label='Dự đoán bước 1', color='blue', linestyle='--')
    plt.plot(filtered_times, filtered_true_values[:, 5], label='Thực tế bước 6', color='red')
    plt.plot(filtered_times, filtered_predictions[:, 5], label='Dự đoán bước 6', color='red', linestyle='--')
    plt.xlabel('Thời gian')
    plt.ylabel('Giá trị GHI_CS')
    plt.title('So sánh thực tế và dự đoán (Bước 1 & 6)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    def calculate_mape(true, pred):
        mask = true != 0
        return np.mean(np.abs((true[mask] - pred[mask]) / true[mask])) * 100 if mask.any() else float('nan')
    
    mape_step1 = calculate_mape(filtered_true_values[:, 0], filtered_predictions[:, 0])
    mape_step6 = calculate_mape(filtered_true_values[:, 5], filtered_predictions[:, 5])
test_start_date = datetime.datetime(2025, 2, 1)
test_end_date = datetime.datetime(2025, 2, 1)
true_values, predictions = test_model(model, scaler, test_start_date, test_end_date)

if true_values is not None and predictions is not None:
    plt.figure(figsize=(15, 6))
    plt.plot(true_values[:, 5], label='Thực tế bước 6', color='red')
    plt.plot(predictions[:, 5], label='Dự đoán bước 6', color='red', linestyle='--')
    plt.xlabel('Thời gian')
    plt.ylabel('Giá trị GHI_CS')
    plt.title('So sánh thực tế và dự đoán (Bước 6)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No valid data available for plotting")
plt.ylabel('Giá trị GHI_CS')
plt.title('So sánh thực tế và dự đoán (Bước 6)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



🔄 Đang xử lý ảnh thành chuỗi dữ liệu multi-channel...


KeyboardInterrupt: 